In [ ]:
#import libraries
#loads state code dictionary for heatmaps  

import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy
from pathlib import Path
import plotly.express as px

code = {'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'}



#This block of code loads both the student population and school finance by state datasets.
#Pulls only 2016 finance data. adds population and revenue per capita rows.

#Student Population Data
student_pop = pd.read_csv(
    Path("../uncc_project1/other_datasets/pop_of_school_kids.csv"))

student_pop = student_pop.dropna()
student_pop.tail()

#School Finance Data

state_school_fin = pd.read_csv(
    Path("../uncc_project1/School_Data/states.csv"))

state_school_fin= state_school_fin.drop('ENROLL', axis=1)
state_school_fin=state_school_fin.drop('OTHER_EXPENDITURE', axis=1)

#SQL stuff to pull only 2016 data
database_connection_string = "sqlite://"
engine = sqlalchemy.create_engine(database_connection_string)

school_db=state_school_fin.to_sql('school_finances', engine, index=False, if_exists='replace')
query_states_2016 = """
SELECT * 
FROM school_finances
WHERE YEAR = 2016
"""

finance_2016=pd.read_sql_query(query_states_2016, con=engine)

#add student population row to finance dataframe 
finance_2016["student_population"] = student_pop["pop_2016"]

#creates revenue per capita row. replace "TOTAL_REVENUE" with a different metric to calculate
#that metric per capita. 
finance_2016["rev_per_capita"]=finance_2016["TOTAL_REVENUE"]/finance_2016["student_population"]

finance_2016_sansDC=finance_2016.drop([8]).reset_index(drop=True)

In [2]:
finance_2016_sansDC.head()

,STATE,YEAR,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,student_population,rev_per_capita
0,Alabama,2016,7498567,797631,4097696,2603240,7708845,3865453,2439634,610743,4860545.0,1.542742
1,Alaska,2016,2494691,307320,1613675,573696,2623014,1251738,983877,255041,741522.0,3.364285
2,Arizona,2016,8503034,1160908,3440726,3901400,7987011,3872250,2913176,528694,6908642.0,1.230782
3,Arkansas,2016,5401016,593656,4122377,684983,5434193,2658253,1802769,518181,2988231.0,1.807429
4,California,2016,89217262,7709079,50904567,30603616,85320133,42587272,26058021,6786142,39296476.0,2.270363


In [3]:
school_rankings= pd.read_csv(
    Path("../uncc_project1/other_datasets/school_rankings_2016.csv"))

school_rank_alphabet=school_rankings.sort_values('State')
school_rank_alphabet['STATE']=school_rank_alphabet['State']
school_rank_alphabet.head()

,Rank,State,College Readiness,High School Graduation Rate,NAEP Math Scores,NAEP Reading Scores,Preschool Enrollment,STATE
44,45,Alabama,44,5,49,48,38,Alabama
48,49,Alaska,37,49,43,49,33,Alaska
46,47,Arizona,47,47,27,37,42,Arizona
37,38,Arkansas,45,9,43,37,19,Arkansas
39,40,California,33,36,38,37,16,California


In [4]:
full_df= pd.merge(
    school_rank_alphabet,
    finance_2016_sansDC,
    how="inner",
    on='STATE',
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [5]:
full_df=full_df.drop(columns=['State'])
full_df=full_df.set_index('STATE')

In [6]:
full_df.head()

,Rank,College Readiness,High School Graduation Rate,NAEP Math Scores,NAEP Reading Scores,Preschool Enrollment,YEAR,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,student_population,rev_per_capita
STATE,,,,,,,,,,,,,,,,,
Alabama,45,44,5,49,48,38,2016,7498567,797631,4097696,2603240,7708845,3865453,2439634,610743,4860545.0,1.542742
Alaska,49,37,49,43,49,33,2016,2494691,307320,1613675,573696,2623014,1251738,983877,255041,741522.0,3.364285
Arizona,47,47,47,27,37,42,2016,8503034,1160908,3440726,3901400,7987011,3872250,2913176,528694,6908642.0,1.230782
Arkansas,38,45,9,43,37,19,2016,5401016,593656,4122377,684983,5434193,2658253,1802769,518181,2988231.0,1.807429
California,40,33,36,38,37,16,2016,89217262,7709079,50904567,30603616,85320133,42587272,26058021,6786142,39296476.0,2.270363


In [7]:
revenue_sorted=full_df.sort_values('rev_per_capita', ascending=False)
revenue_sorted.head()

,Rank,College Readiness,High School Graduation Rate,NAEP Math Scores,NAEP Reading Scores,Preschool Enrollment,YEAR,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,student_population,rev_per_capita
STATE,,,,,,,,,,,,,,,,,
Wyoming,39,43,40,9,13,47,2016,2044669,123012,1175899,745758,2034229,921494,585700,454302,584910.0,3.495698
Vermont,5,12,29,5,4,2,2016,2112365,111891,1614518,385956,2082696,1013209,583178,42911,623354.0,3.388709
New York,19,11,37,27,30,6,2016,66912661,3369803,27437593,36105265,68282026,43964520,15883500,4656189,19836286.0,3.373245
Alaska,49,37,49,43,49,33,2016,2494691,307320,1613675,573696,2623014,1251738,983877,255041,741522.0,3.364285
New Jersey,1,3,2,2,2,1,2016,30012666,1187430,11681799,17143437,29665476,15831343,9549742,1414861,8978416.0,3.342757


In [8]:
revenue_sorted.iloc[0:10].hvplot.bar(
    x='STATE',
    y='rev_per_capita',
    rot=45,
    xlabel='',
    ylabel = 'Revenue per Student, USD',
    title= 'Top 10 States in Total School Revenue per Capita',
    height=400,
    width=700,
)

:Bars   [STATE]   (rev_per_capita)

In [9]:
rank_sorted=full_df.sort_values('Rank')
rank_sorted.head(10)

,Rank,College Readiness,High School Graduation Rate,NAEP Math Scores,NAEP Reading Scores,Preschool Enrollment,YEAR,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,student_population,rev_per_capita
STATE,,,,,,,,,,,,,,,,,
New Jersey,1,3,2,2,2,1,2016,30012666,1187430,11681799,17143437,29665476,15831343,9549742,1414861,8978416.0,3.342757
Massachusetts,2,4,16,1,1,5,2016,17484704,765043,6770086,9949575,17037880,9991819,4970289,945525,6823721.0,2.562342
Connecticut,3,1,13,9,2,3,2016,11419673,468595,4411204,6539874,10826431,6110631,3307162,620905,3587685.0,3.183020
New Hampshire,4,8,11,5,4,7,2016,3150473,169166,1004753,1976554,3124778,1743022,959654,137180,1335015.0,2.359878
Vermont,5,12,29,5,4,2,2016,2112365,111891,1614518,385956,2082696,1013209,583178,42911,623354.0,3.388709
Illinois,6,2,24,22,13,8,2016,32908958,2331724,12053316,18523918,33037244,17804976,10426170,2015647,12835726.0,2.563856
Colorado,7,6,44,16,6,12,2016,10123271,713914,4365019,5044338,9878524,4786838,3333043,880204,5530105.0,1.830575
Wisconsin,8,32,8,4,6,39,2016,11697466,782610,5986763,4928093,11787535,5760418,3691439,1082849,5772917.0,2.026266
Indiana,9,13,14,9,10,43,2016,12732161,954837,7780711,3996613,11535287,5730602,3663349,986746,6634007.0,1.919226


In [10]:
revenue_sorted.iloc[39:49].hvplot.bar(
    x='STATE',
    y='rev_per_capita',
    rot=45,
    xlabel='',
    ylabel = 'Revenue per Student, USD',
    title= 'Bottom 10 States in Total School Revenue per Capita',
    height=400,
    width=700,
)

:Bars   [STATE]   (rev_per_capita)

In [36]:
full_df['rev_per_capita'].hvplot.hist()

:Histogram   [rev_per_capita]   (rev_per_capita_count)

In [ ]:
percent_of_gdp = full_df['